In [1]:
import tqdm

def nop(it, *a, **k):
    return it

real_tqdm = tqdm.tqdm
tqdm.tqdm = nop

import time
import os
import glob
import pickle

import numpy as np
np.bool = np.bool_
import cv2
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch.nn.functional as F

from utils.inference.image_processing import crop_face, get_final_image, show_images, normalize_and_torch, normalize_and_torch_batch
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement, crop_frames_and_get_transforms, resize_frames
from utils.inference.core import model_inference, transform_target_to_torch
from utils.inference.faceshifter_run import faceshifter_batch, faceshifter_batch_zattrs
from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:143: DeprecationWarning: In accordance with NEP 32, the function mirr was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  mirr = onp.mirr
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:160: DeprecationWarning: In accordance with NEP 32, the function npv was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  npv = onp.npv
/compuworks/anaconda3/envs/xaise

### Load Models

In [2]:
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)


[11:14:23] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.5.0. Attempting to upgrade...
[11:14:23] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[11:14:23] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8204 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


Network [LIPSPADEGenerator] was created. Total number of parameters: 72.2 million. To see the architecture, do print(network).
Load checkpoint from path:  weights/10_net_G.pth


In [3]:
class DoublePCA:
    def __init__(self, layer_num=8, root_path="./pca_pkl"):
        self.layer_num = layer_num

        self.pca1_list = []
        for z_i in range(self.layer_num):
            with open(f"{root_path}/Altered_zattr{z_i}PCA.pkl", "rb") as file:
                self.pca1_list.append(pickle.load(file))
        with open(f"{root_path}/Altered_zattr_doublePCA.pkl", "rb") as file:
            self.pca2 = pickle.load(file)
        with open(f"{root_path}/Altered_zattr_doublePCAMinMax.pkl", "rb") as file:
            minmax = pickle.load(file)
        self.pca2_min = minmax["min"]
        self.pca2_max = minmax["max"]
    def transform(self, z_embeds):
        p1emb_array = [self.pca1_list[z_i].transform(z_embeds[z_i]) for z_i in range(self.layer_num)]
        p1emb_array = np.concatenate(p1emb_array, axis=1)
        return self.pca2.transform(p1emb_array)
    def inverse_transform(self, p2emb_array):
        p1emb_array = self.pca2.inverse_transform(p2emb_array).reshape([-1, self.layer_num, 128])
        z_embeds = [self.pca1_list[z_i].inverse_transform(p1emb_array[:,z_i]) for z_i in range(self.layer_num)]
        return z_embeds
    def calcul_z_embed_diff(self, target, original):
        assert target.shape[0] == original.shape[0]
        target = self.inverse_transform(target)
        original = self.inverse_transform(original)
        return [target[i]-original[i] for i in range(self.layer_num)]

### Set here path to source image and video for faceswap

# 특징 빼기 + 인젝션

# pca

In [10]:
from insightface.utils import face_align
import ipywidgets as widgets
import copy

crop_size = 224

set_target = False
half=True
similarity_th=0.15

crop_frames_list = None
target_batch_rs = None
image_to_image = True
start_id = 50

pca = DoublePCA()


pca_min = pca.pca2_min
pca_max = pca.pca2_max

z_embed = None
original_shape = None
pca_array = None
orig_z_embed = None
original_pca = None

def set_target(path_to_target='examples/images/jaeseung.jpg', image_to_image=True):
    global z_embed, zattr_id, original_shape, pca_array, orig_z_embed, original_pca
    if image_to_image:
        target_full = cv2.imread(path_to_target)
        full_frames = [target_full]
    else:
        full_frames, fps = read_video(path_to_video)
    target = get_target(full_frames, app, crop_size)

    target_norm = normalize_and_torch_batch(np.array(target))
    target_embeds = netArc(F.interpolate(target_norm, scale_factor=0.5, mode='bilinear', align_corners=True))

    # Get the cropped faces from original frames and transformations to get those crops
    crop_frames_list, tfm_array_list = crop_frames_and_get_transforms(full_frames,
                                                                    target_embeds,
                                                                    app,
                                                                    netArc,
                                                                    crop_size,
                                                                    set_target,
                                                                    similarity_th=similarity_th
                                                                    )
    resized_frs, present = resize_frames(crop_frames_list[0])
    resized_frs = np.array(resized_frs)

    target_batch_rs = transform_target_to_torch(resized_frs, half=True)

    zattrs = G.get_attr(target_batch_rs)
    z_embed = list(zattrs)
    original_shape = [attr.shape for attr in z_embed]

    for i in range(len(z_embed)):
        z_embed[i] = z_embed[i].detach().cpu().numpy().reshape(1, -1)
    

    original_pca = pca.transform(z_embed)
    pca_array = original_pca.copy()
    orig_z_embed = copy.deepcopy(z_embed)



path_to_target="examples/images/elon_musk.jpg"

set_target(path_to_target)




source_pic = "examples/images/junseok.png"

source_full = cv2.imread(source_pic)
full_frames = [source_full]
source = crop_face(source_full, app, crop_size)[0][:,:,::-1]
source_curr = source
source_curr = normalize_and_torch(source_curr)
source_embed = netArc(F.interpolate(source_curr, scale_factor=0.5, mode='bilinear', align_corners=True)).half()

source_id = -1 if source_pic else 3
if source_id >= 0: 
    source_embed = torch.from_numpy(np.array([np.load("./examples/arc_embeds/{}.npy".format(source_id))])).half().to("cuda")


def inject_drawing():
    global crop_frames_list, pca_array, z_embed, source_embed
    plt.figure(num=1, clear=True, figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.imread(path_to_target)[:,:,::-1])
    plt.title("Target Face")
    plt.subplot(1, 2, 2)

    z_embed_diff = pca.calcul_z_embed_diff(pca_array, original_pca)
    for zattr_id in range(len(z_embed)):
        z_embed[zattr_id] = torch.from_numpy((orig_z_embed[zattr_id]+z_embed_diff[zattr_id]).reshape(*original_shape[zattr_id])).half().to("cuda")
    #source_embed[0][0] = 10.0

    Y_st = faceshifter_batch_zattrs(source_embed, z_embed, 1, G)
    #Y_st = faceshifter_batch(source_embed, target_batch_rs[i:i+BS], G)

    torch.cuda.empty_cache()

    plt.imshow(Y_st[0][:,:,::-1])
    plt.title("After Swap")
    plt.show()

pca_sliders = []

iact_plot = widgets.interactive(
    inject_drawing
)

for i in range(40):
    pca_slider = widgets.FloatSlider(
        value=pca_array[0, i], 
        min = pca_min[i],
        max = pca_max[i],
        description=f"PCA #{i}",
        continuous_update=False,
        layout=widgets.Layout(width="300px"),
    )
    pca_slider.idx = i
    pca_slider.observe(lambda change: exec("pca_array[0, change.owner.idx]=change.new;iact_plot.update()"), names="value")
    #pca_slider.observe(lambda change: print(change.owner.idx), names="value")
    pca_sliders.append(pca_slider)
    

slider_multibox = widgets.HBox(
    children=[
    widgets.VBox(
    children = pca_sliders[:20]),
    widgets.VBox(
    children = pca_sliders[20:]),
    ]
)


display(iact_plot, slider_multibox)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

# PCA 값에 따른 변화 그림

In [11]:
# 재미용


pca = DoublePCA()
pca_min = pca.pca2_min
pca_max = pca.pca2_max
pca_orig_range = pca_max - pca_min
pca_min -= pca_orig_range*0.3
pca_max += pca_orig_range*0.3

set_target = False
half=True
similarity_th=0.15
crop_size=224



#start_id = 0
#tmp_embed = [np.load("./embeds/{}.npy".format(start_id))]
#pca_array = pca.transform(tmp_embed)

z_embed = None
original_shape = None
pca_array = None
orig_z_embed = None
original_pca = None

def set_target(path_to_target='examples/images/jaeseung.jpg', image_to_image=True):
    global z_embed, original_shape, pca_array, orig_z_embed, original_pca, full_frames
    if image_to_image:
        target_full = cv2.imread(path_to_target)
        full_frames = [target_full]
    else:
        full_frames, fps = read_video(path_to_video)
    target = get_target(full_frames, app, crop_size)

    target_norm = normalize_and_torch_batch(np.array(target))
    target_embeds = netArc(F.interpolate(target_norm, scale_factor=0.5, mode='bilinear', align_corners=True))

    # Get the cropped faces from original frames and transformations to get those crops
    crop_frames_list, tfm_array_list = crop_frames_and_get_transforms(full_frames,
                                                                    target_embeds,
                                                                    app,
                                                                    netArc,
                                                                    crop_size,
                                                                    set_target,
                                                                    similarity_th=similarity_th
                                                                    )
    resized_frs, present = resize_frames(crop_frames_list[0])
    resized_frs = np.array(resized_frs)

    target_batch_rs = transform_target_to_torch(resized_frs, half=True)

    zattrs = G.get_attr(target_batch_rs)
    z_embed = list(zattrs)
    original_shape = [attr.shape for attr in z_embed]

    for i in range(len(z_embed)):
        z_embed[i] = z_embed[i].detach().cpu().numpy().reshape(1, -1)
    

    original_pca = pca.transform(z_embed)
    pca_array = original_pca.copy()
    orig_z_embed = copy.deepcopy(z_embed)



set_target(path_to_target="examples/images/jaeseung_3.jpg")




source_pic = "examples/images/elon_musk.jpg"

source_full = cv2.imread(source_pic)
source = crop_face(source_full, app, crop_size)[0][:,:,::-1]
source_curr = source
source_curr = normalize_and_torch(source_curr)
source_embed = netArc(F.interpolate(source_curr, scale_factor=0.5, mode='bilinear', align_corners=True)).half()


source_id = -1 if source_pic else 3
#source_embed = torch.from_numpy(np.array([np.load("./examples/arc_embeds/{}.npy".format(source_id))])).half().to("cuda")

num_plotted_pca = 20
n_ticks = 5

plt.figure(num=1, clear=True, figsize=(n_ticks*2, int((num_plotted_pca*3+1)*1.6)))


plt.subplot(num_plotted_pca+1,n_ticks, 1)
plt.imshow(cv2.imread(source_pic)[:,:,::-1])
plt.title("Target Face")
plt.axis("off")
plt.subplot(num_plotted_pca+1,n_ticks,2)

tmp_z_embed = [torch.from_numpy(z_embed[i].reshape(original_shape[i])).half().to("cuda") for i in range(len(z_embed))]

Y_st = faceshifter_batch_zattrs(source_embed, tmp_z_embed, 1, G)
torch.cuda.empty_cache()

plt.imshow(Y_st[0])
plt.title("Swapped (origin)")
#plt.axis("off")
for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
plt.xticks([])
plt.yticks([])



for pci_i in range(num_plotted_pca):
    interval = (pca_max[pci_i] - pca_min[pci_i]) / (n_ticks-1)
    for c in range(n_ticks):
        plt.subplot(num_plotted_pca+1,n_ticks, (pci_i+1)*n_ticks+c+1)
        new_pca_array = pca_array.copy()
        new_pca_array[0, pci_i] = pca_min[pci_i] + (interval * c)

        z_embed_diff = pca.calcul_z_embed_diff(new_pca_array, original_pca)
        for zattr_id in range(len(z_embed)):
            z_embed[zattr_id] = torch.from_numpy((orig_z_embed[zattr_id]+z_embed_diff[zattr_id]).reshape(*original_shape[zattr_id])).half().to("cuda")
        #source_embed[0][0] = 10.0

        Y_st = faceshifter_batch_zattrs(source_embed, z_embed, 1, G)
        #Y_st = faceshifter_batch(source_embed, target_batch_rs[i:i+BS], G)
        torch.cuda.empty_cache()
        
        plt.imshow(Y_st[0][:,:,::-1])
        #plt.title("Swapped Face ")
        #plt.axis("off")
        for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")

        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))

plt.tight_layout()
save_path = f"./pca_figs/doublePCA_for_zattr_tgt{source_id}_to{num_plotted_pca}_tmp.png"
plt.savefig(save_path)
plt.close()
print(f"saved at {save_path}")

saved at ./pca_figs/doublePCA_for_zattr_tgt-1_to20_tmp.png


# zattr[i]

In [ ]:


set_target = False
half=True
similarity_th=0.15

crop_frames_list = None
target_batch_rs = None

for zattr_id in [1, 2, 3, 4, 5, 6, 7]: 

    with open(f"zattr{zattr_id}PCA.pkl", "rb") as file:
        pca = pickle.load(file)



    with open(f"zattr{zattr_id}PCAMinMax.pkl", "rb") as file:
        pca_minmax = pickle.load(file)
    pca_min = pca_minmax["min"]
    pca_max = pca_minmax["max"]
    pca_orig_range = pca_max - pca_min
    pca_min -= pca_orig_range*0.1
    pca_max += pca_orig_range*0.1

    #tmp_embed = [np.load("./embeds/{}.npy".format(start_id))]
    #pca_array = pca.transform(tmp_embed)

    start_id = 10
    with open("./examples/z_embeds/VggFace2-crop/{}.pkl".format(start_id), "rb") as file:
        z_embed = pickle.load(file)
    original_shape = [attr.shape for attr in z_embed]
    pca_array = pca.transform(z_embed[zattr_id].reshape(1, -1))
    orig_z_embed = z_embed[zattr_id]
    for i in range(len(z_embed)):
        z_embed[i] = torch.from_numpy(z_embed[i]).half().to("cuda")
        
    source_id = 3
    source_embed = torch.from_numpy(np.array([np.load("./examples/arc_embeds/{}.npy".format(source_id))])).half().to("cuda")


    num_plotted_pca = 40
    n_ticks = 5

    plt.figure(num=1, clear=True, figsize=(n_ticks*2, int(num_plotted_pca*1.75)) )
    plt.subplot(num_plotted_pca+1,n_ticks, 1)
    plt.imshow(cv2.imread("./examples/images/VggFace2-crop/n000007/0112_01.jpg")[:,:,::-1])
    plt.title("Target Face")
    plt.axis("off")
    plt.subplot(num_plotted_pca+1,n_ticks,2)

    Y_st = faceshifter_batch_zattrs(source_embed, z_embed, 1, G)
    torch.cuda.empty_cache()

    plt.imshow(Y_st[0][:, :,])
    plt.title("Swapped (origin)")
    #plt.axis("off")
    for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
    plt.xticks([])
    plt.yticks([])



    for pci_i in range(num_plotted_pca):
        interval = (pca_max[pci_i] - pca_min[pci_i]) / (n_ticks-1)
        for c in range(n_ticks):
            plt.subplot(num_plotted_pca+1,n_ticks, (pci_i+1)*n_ticks+c+1)
            new_pca_array = pca_array.copy()
            before_modi_embed = pca.inverse_transform(new_pca_array) 
            new_pca_array[0, pci_i] = pca_min[pci_i] + (interval * c)
            modified_embed = pca.inverse_transform(new_pca_array)
            embed_diff =  modified_embed - before_modi_embed
            z_embed[zattr_id] = torch.from_numpy(orig_z_embed+embed_diff.reshape(*original_shape[zattr_id])).half().to("cuda")
            #source_embed[0][0] = 10.0

            Y_st = faceshifter_batch_zattrs(source_embed, z_embed, 1, G)
            #Y_st = faceshifter_batch(source_embed, target_batch_rs[i:i+BS], G)

            plt.imshow(Y_st[0])
            #plt.title("Swapped Face ")
            #plt.axis("off")
            for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
            plt.xticks([])
            plt.yticks([])
            plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")

            if c == 0:
                plt.ylabel("PCA #{}".format(pci_i))

    save_path = f"./pca_figs/PCA_for_zattr[{zattr_id}]_tgt{source_id}_to{num_plotted_pca}.png"
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"saved at {save_path}")

In [9]:
# for comparison

set_target = False
half=True
similarity_th=0.15
crop_size = 224 # don't change this
BS = 60


pca = DoublePCA()
pca_min = pca.pca2_min
pca_max = pca.pca2_max
pca_orig_range = pca_max - pca_min
pca_min -= pca_orig_range*0.3
pca_max += pca_orig_range*0.3



source_pic = "examples/images/junseok.png"

source_full = cv2.imread(source_pic)
full_frames = [source_full]
source = crop_face(source_full, app, crop_size)[0][:,:,::-1]
source_curr = source
source_curr = normalize_and_torch(source_curr)
source_embed = netArc(F.interpolate(source_curr, scale_factor=0.5, mode='bilinear', align_corners=True)).half()

source_id = -1 if source_pic else 3
if source_id >= 0: 
    source_embed = torch.from_numpy(np.array([np.load("./examples/arc_embeds/{}.npy".format(source_id))])).half().to("cuda")


z_embed = None
original_shape = None
pca_array = None
orig_z_embed = None
original_pca = None
crop_frames_list = None
def set_target(path_to_target='examples/images/jaeseung.jpg', image_to_image=True):
    global z_embed, zattr_id, original_shape, pca_array, orig_z_embed, original_pca, crop_frames_list
    if image_to_image:
        target_full = cv2.imread(path_to_target)
        full_frames = [target_full]
    else:
        full_frames, fps = read_video(path_to_video)
    target = get_target(full_frames, app, crop_size)

    target_norm = normalize_and_torch_batch(np.array(target))
    target_embeds = netArc(F.interpolate(target_norm, scale_factor=0.5, mode='bilinear', align_corners=True))

    # Get the cropped faces from original frames and transformations to get those crops
    crop_frames_list, tfm_array_list = crop_frames_and_get_transforms(full_frames,
                                                                    target_embeds,
                                                                    app,
                                                                    netArc,
                                                                    crop_size,
                                                                    set_target,
                                                                    similarity_th=similarity_th
                                                                    )
    resized_frs, present = resize_frames(crop_frames_list[0])
    resized_frs = np.array(resized_frs)

    target_batch_rs = transform_target_to_torch(resized_frs, half=True)

    zattrs = G.get_attr(target_batch_rs)
    z_embed = list(zattrs)
    original_shape = [attr.shape for attr in z_embed]

    for i in range(len(z_embed)):
        z_embed[i] = z_embed[i].detach().cpu().numpy().reshape(1, -1)
    

    original_pca = pca.transform(z_embed)
    pca_array = original_pca.copy()
    orig_z_embed = copy.deepcopy(z_embed)


set_target(path_to_target="examples/images/jaeseung.jpg")

num_plotted_pca = 20
n_ticks = 7




plt.figure(num=1, clear=True, figsize=(n_ticks*2, int((num_plotted_pca*3+1)*1.6)) )

def draw_top(subplot_tuple, img):
    plt.subplot(*subplot_tuple)
    plt.imshow(img)
    plt.title("Target Face")
    plt.axis("off")

def draw_swap(subplot_tuple, new_pca_array, is_top=False):
    global target_batch_rs, G
    plt.subplot(*subplot_tuple)

    z_embed_diff = pca.calcul_z_embed_diff(new_pca_array, original_pca)
    for zattr_id in range(len(z_embed)):
        z_embed[zattr_id] = torch.from_numpy((orig_z_embed[zattr_id]+z_embed_diff[zattr_id]).reshape(*original_shape[zattr_id])).half().to("cuda")

    Y_st = faceshifter_batch_zattrs(source_embed, z_embed, 1, G)
    torch.cuda.empty_cache()
    plt.imshow(Y_st[0][:,:,::-1])
    
    for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
    plt.xticks([])
    plt.yticks([])
    if is_top:
        plt.title("Swapped (Random)")


set_target(path_to_target="examples/images/jaeseung.jpg")
draw_top((num_plotted_pca*3+1, n_ticks, 1), crop_frames_list[0][0][:,:,::-1])
draw_swap((num_plotted_pca*3+1, n_ticks, 2), pca_array, is_top=True)

set_target(path_to_target="examples/images/elon_musk.jpg")
draw_top((num_plotted_pca*3+1, n_ticks, 3), crop_frames_list[0][0][:,:,::-1])
draw_swap((num_plotted_pca*3+1, n_ticks, 4), pca_array, is_top=True)

set_target(path_to_target="examples/images/tgt2.png")
draw_top((num_plotted_pca*3+1, n_ticks, 5), crop_frames_list[0][0][:,:,::-1])
draw_swap((num_plotted_pca*3+1, n_ticks, 6), pca_array, is_top=True)

for pci_i in range(num_plotted_pca):
    interval = (pca_max[pci_i] - pca_min[pci_i]) / (n_ticks-1)
    for c in range(n_ticks):
        
        set_target(path_to_target="examples/images/jaeseung.jpg")
        new_pca_array = pca_array.copy()
        new_pca_array[0, pci_i] = pca_min[pci_i] + (interval * c)
        draw_swap((num_plotted_pca*3+1, n_ticks, (pci_i*3+1)*n_ticks+c+1), new_pca_array)
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")
        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))
        set_target(path_to_target="examples/images/elon_musk.jpg")
        new_pca_array = pca_array.copy()
        new_pca_array[0, pci_i] = pca_min[pci_i] + (interval * c)
        draw_swap((num_plotted_pca*3+1, n_ticks, (pci_i*3+2)*n_ticks+c+1), new_pca_array)
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")
        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))
        set_target(path_to_target="examples/images/tgt2.png")
        new_pca_array = pca_array.copy()
        new_pca_array[0, pci_i] = pca_min[pci_i] + (interval * c)
        draw_swap((num_plotted_pca*3+1, n_ticks, (pci_i*3+3)*n_ticks+c+1), new_pca_array)
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")
        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))

plt.tight_layout()
save_path = f"./pca_figs/doublePCA_for_zattr_src{source_id}_Triple_to{num_plotted_pca}_tmp.png"
plt.savefig(save_path)
plt.close()
print(f"saved at {save_path}")

saved at ./pca_figs/doublePCA_for_zattr_src-1_Triple_to20_tmp.png
